# Miniverse Tutorial: Building Agent-Based Simulations from Scratch

This notebook teaches you how to use Miniverse by building up from basic primitives to full simulations.

**What you'll learn:**
- Core data structures (AgentProfile, WorldState, Plan, Scratchpad, Memory)
- Cognition modules (Executor, Planner, ReflectionEngine)
- Building simulations step-by-step
- Understanding what's happening "under the hood"

**Prerequisites:**
- Python 3.10+
- Miniverse installed (`uv sync`)
- Optional: LLM API key for later sections

## Part 1: Core Primitives - Data Structures

Before running any simulation, let's understand the basic building blocks.

### 1.1 The Stat Object - Flexible Metrics

`Stat` is the basic unit for representing any numeric value with metadata.

In [1]:
from miniverse import Stat

# Create different types of stats
energy = Stat(value=80, unit="%", label="Energy Level")
backlog = Stat(value=15, label="Task Backlog")
temperature = Stat(value=22.5, unit="°C", label="Room Temperature")

print("Energy stat:")
print(f"  Value: {energy.value}")
print(f"  Unit: {energy.unit}")
print(f"  Label: {energy.label}")
print(f"\nBacklog: {backlog.value} {backlog.label}")
print(f"Temperature: {temperature.value}{temperature.unit}")

Energy stat:
  Value: 80
  Unit: %
  Label: Energy Level

Backlog: 15 Task Backlog
Temperature: 22.5°C


### 1.2 AgentProfile - Who is this agent?

Every agent has a profile describing their identity, personality, and goals.

In [2]:
from miniverse import AgentProfile

# Create an agent profile
alice = AgentProfile(
    agent_id="alice",
    name="Alice",
    age=28,
    background="Software engineer who loves collaborative work",
    role="developer",
    personality="friendly and detail-oriented",
    skills={"python": "expert", "teamwork": "proficient"},
    goals=["Write clean code", "Help teammates", "Maintain work-life balance"],
    relationships={"bob": "colleague"}
)

print("Agent Profile:")
print(f"  ID: {alice.agent_id}")
print(f"  Name: {alice.name}, Age: {alice.age}")
print(f"  Role: {alice.role}")
print(f"  Personality: {alice.personality}")
print(f"  Background: {alice.background}")
print(f"  Skills: {alice.skills}")
print(f"  Goals: {alice.goals}")
print(f"  Relationships: {alice.relationships}")

Agent Profile:
  ID: alice
  Name: Alice, Age: 28
  Role: developer
  Personality: friendly and detail-oriented
  Background: Software engineer who loves collaborative work
  Skills: {'python': 'expert', 'teamwork': 'proficient'}
  Goals: ['Write clean code', 'Help teammates', 'Maintain work-life balance']
  Relationships: {'bob': 'colleague'}


### 1.3 AgentStatus - Current state of an agent

While `AgentProfile` describes WHO the agent is, `AgentStatus` tracks their CURRENT state.

In [3]:
from miniverse import AgentStatus

# Create agent status (what's happening RIGHT NOW)
alice_status = AgentStatus(
    agent_id="alice",
    display_name="Alice (Developer)",
    activity="coding",
    location="office_2b",
    attributes={
        "energy": Stat(value=75, unit="%", label="Energy"),
        "focus": Stat(value=85, unit="%", label="Focus"),
        "happiness": Stat(value=70, unit="%", label="Happiness")
    }
)

print("Agent Status (Current State):")
print(f"  Agent: {alice_status.display_name}")
print(f"  Activity: {alice_status.activity}")
print(f"  Location: {alice_status.location}")
print(f"  Attributes:")
for key, stat in alice_status.attributes.items():
    print(f"    - {stat.label}: {stat.value}{stat.unit or ''}")

Agent Status (Current State):
  Agent: Alice (Developer)
  Activity: coding
  Location: office_2b
  Attributes:
    - Energy: 75%
    - Focus: 85%
    - Happiness: 70%


### 1.4 WorldState - The complete simulation state

`WorldState` contains everything: resources, environment, and all agents.

In [4]:
from miniverse import WorldState, ResourceState, EnvironmentState
from datetime import datetime, timezone

# Create a simple world state
world = WorldState(
    tick=0,
    timestamp=datetime.now(timezone.utc),
    environment=EnvironmentState(
        metrics={
            "temperature": Stat(value=22.0, unit="°C", label="Room Temperature")
        }
    ),
    resources=ResourceState(
        metrics={
            "task_backlog": Stat(value=10, label="Tasks Pending"),
            "coffee": Stat(value=5, unit="cups", label="Coffee Remaining")
        }
    ),
    agents=[alice_status]
)

print("World State:")
print(f"  Tick: {world.tick}")
print(f"  Timestamp: {world.timestamp}")
print(f"\n  Environment:")
for key, stat in world.environment.metrics.items():
    print(f"    - {stat.label}: {stat.value}{stat.unit or ''}")
print(f"\n  Resources:")
for key, stat in world.resources.metrics.items():
    print(f"    - {stat.label}: {stat.value} {stat.unit or ''}")
print(f"\n  Agents: {len(world.agents)}")
for agent in world.agents:
    print(f"    - {agent.display_name} ({agent.activity})")

World State:
  Tick: 0
  Timestamp: 2025-10-16 00:10:28.413400+00:00

  Environment:
    - Room Temperature: 22.0°C

  Resources:
    - Tasks Pending: 10 
    - Coffee Remaining: 5 cups

  Agents: 1
    - Alice (Developer) (coding)


### 1.5 Plan - Multi-step agent plans

Agents can have plans with multiple steps. Let's see what a plan looks like.

In [ ]:
from miniverse.cognition import Plan, PlanStep

# Create a multi-step plan
morning_plan = Plan(
    steps=[
        PlanStep(description="Review overnight alerts", metadata={"duration": 1}),
        PlanStep(description="Work on feature implementation", metadata={"duration":
3}),
        PlanStep(description="Code review for teammate", metadata={"duration": 1}),
        PlanStep(description="Take lunch break", metadata={"duration": 1})
    ]
)

print("Agent Plan:")
print(f"  Total steps: {len(morning_plan.steps)}")
print(f"\n  Steps:")
for i, step in enumerate(morning_plan.steps):
    duration = step.metadata.get("duration", "?")
    marker = "→" if i == 0 else " "  # First step is current
    print(f"    {marker} Step {i}: {step.description} ({duration} ticks)")

print(f"\nNote: Plan progress is tracked by the orchestrator in scratchpad.")
print(f"The orchestrator advances through steps automatically.")

Agent Plan:
  Total steps: 4


AttributeError: 'Plan' object has no attribute 'current_step_index'

### 1.6 Scratchpad - Working memory

The scratchpad is a simple key-value store for agents to track temporary state.

In [ ]:
from miniverse.cognition import Scratchpad

# Create a scratchpad and use it
scratchpad = Scratchpad()

# Store various working memory items
scratchpad.state["last_break"] = 5  # tick number
scratchpad.state["current_task"] = "implement_authentication"
scratchpad.state["blockers"] = ["waiting for API key", "database schema unclear"]
scratchpad.state["energy_trend"] = "declining"

print("Scratchpad (Working Memory):")
for key, value in scratchpad.state.items():
    print(f"  {key}: {value}")

# Access specific values
print(f"\nLast break was at tick: {scratchpad.state.get('last_break')}")
print(f"Current blockers: {scratchpad.state.get('blockers')}")

### 1.7 AgentMemory - Long-term memory entries

Memories have importance scores, tags, and timestamps.

In [ ]:
from miniverse import AgentMemory
from datetime import datetime, timezone

# Create some memory entries
memories = [
    AgentMemory(
        agent_id="alice",
        tick=5,
        timestamp=datetime.now(timezone.utc),
        content="Helped Bob debug a tricky concurrency issue. He seemed grateful.",
        memory_type="observation",
        importance=7,
        tags=["collaboration", "teamwork", "technical"]
    ),
    AgentMemory(
        agent_id="alice",
        tick=8,
        timestamp=datetime.now(timezone.utc),
        content="Feeling energized after completing the authentication feature.",
        memory_type="observation",
        importance=5,
        tags=["achievement", "energy"]
    ),
    AgentMemory(
        agent_id="alice",
        tick=10,
        timestamp=datetime.now(timezone.utc),
        content="I've been helping teammates a lot - this aligns with my goal of collaboration.",
        memory_type="reflection",
        importance=8,
        tags=["reflection", "goals", "teamwork"]
    )
]

print("Agent Memories:")
for i, memory in enumerate(memories, 1):
    print(f"\n  Memory {i} (Tick {memory.tick}):")
    print(f"    Type: {memory.memory_type}")
    print(f"    Importance: {memory.importance}/10")
    print(f"    Tags: {', '.join(memory.tags)}")
    print(f"    Content: \"{memory.content}\"")

## Part 2: Cognition Modules

Now let's explore the modules that power agent decision-making.

### 2.1 Executor - The decision-maker

Every agent needs an executor to choose actions. Let's build a simple one.

In [ ]:
from miniverse import AgentAction, Perception

# Define a simple executor
class AlwaysWorkExecutor:
    """Simplest possible executor - always chooses 'work'."""
    
    async def choose_action(self, agent_id, perception, scratchpad, *, plan, plan_step, context):
        return AgentAction(
            agent_id=agent_id,
            tick=perception.tick,
            action_type="work",
            reasoning="I always work - it's hardcoded!"
        )

# Test it
executor = AlwaysWorkExecutor()

# Create a mock perception
mock_perception = Perception(
    tick=1,
    agent_id="alice",
    visible_resources={"task_backlog": Stat(value=10)},
    personal_attributes={"energy": Stat(value=80, unit="%")},
    nearby_agents=[],
    recent_messages=[]
)

# Get action
import asyncio
action = await executor.choose_action(
    agent_id="alice",
    perception=mock_perception,
    scratchpad=None,
    plan=None,
    plan_step=None,
    context={}
)

print("Executor Output:")
print(f"  Agent: {action.agent_id}")
print(f"  Action: {action.action_type}")
print(f"  Reasoning: {action.reasoning}")

### 2.2 A smarter executor - Threshold logic

Let's build an executor that makes decisions based on agent state.

In [ ]:
class ThresholdExecutor:
    """Makes decisions based on energy and backlog thresholds."""
    
    async def choose_action(self, agent_id, perception, scratchpad, *, plan, plan_step, context):
        # Extract current state
        energy_stat = perception.personal_attributes.get("energy")
        energy = float(energy_stat.value) if energy_stat else 100
        
        backlog_stat = perception.visible_resources.get("task_backlog")
        backlog = float(backlog_stat.value) if backlog_stat else 0
        
        # Decision logic
        if energy < 30:
            action_type = "rest"
            reasoning = f"Energy low ({energy}%) - need to rest"
        elif backlog > 8:
            action_type = "work"
            reasoning = f"High backlog ({int(backlog)} tasks) - must work"
        else:
            action_type = "rest"
            reasoning = f"Energy {energy}%, backlog {int(backlog)} - resting"
        
        return AgentAction(
            agent_id=agent_id,
            tick=perception.tick,
            action_type=action_type,
            reasoning=reasoning
        )

# Test with different states
executor = ThresholdExecutor()

test_cases = [
    {"energy": 80, "backlog": 10, "desc": "High energy, high backlog"},
    {"energy": 25, "backlog": 10, "desc": "Low energy, high backlog"},
    {"energy": 80, "backlog": 3, "desc": "High energy, low backlog"},
]

print("Threshold Executor Tests:\n")
for case in test_cases:
    perception = Perception(
        tick=1,
        agent_id="alice",
        visible_resources={"task_backlog": Stat(value=case["backlog"])},
        personal_attributes={"energy": Stat(value=case["energy"], unit="%")},
        nearby_agents=[],
        recent_messages=[]
    )
    
    action = await executor.choose_action(
        agent_id="alice",
        perception=perception,
        scratchpad=None,
        plan=None,
        plan_step=None,
        context={}
    )
    
    print(f"  {case['desc']}:")
    print(f"    → Action: {action.action_type}")
    print(f"    → Reasoning: {action.reasoning}")
    print()

### 2.3 AgentCognition - Bundling modules together

The `AgentCognition` object bundles executor, planner, reflection, and scratchpad.

In [ ]:
from miniverse.cognition import AgentCognition

# Minimal cognition: just executor
cognition_minimal = AgentCognition(
    executor=ThresholdExecutor()
)

print("Minimal Cognition:")
print(f"  Executor: {type(cognition_minimal.executor).__name__}")
print(f"  Planner: {cognition_minimal.planner}")
print(f"  Reflection: {cognition_minimal.reflection}")
print(f"  Scratchpad: {cognition_minimal.scratchpad}")

# Cognition with scratchpad
cognition_with_memory = AgentCognition(
    executor=ThresholdExecutor(),
    scratchpad=Scratchpad()
)

print("\nCognition with Scratchpad:")
print(f"  Executor: {type(cognition_with_memory.executor).__name__}")
print(f"  Scratchpad: {type(cognition_with_memory.scratchpad).__name__}")

# Store something in scratchpad
cognition_with_memory.scratchpad.state["test_key"] = "test_value"
print(f"  Scratchpad state: {cognition_with_memory.scratchpad.state}")

## Part 3: First Simulation - Putting it together

Let's run our first actual simulation with all the pieces we've learned.

### 3.1 Define physics (SimulationRules)

Physics determine how the world changes each tick.

In [ ]:
from miniverse import SimulationRules, WorldState, AgentAction

class SimpleWorkshopRules(SimulationRules):
    """Basic workshop physics: working drains energy, resting recovers it."""
    
    def apply_tick(self, state: WorldState, tick: int) -> WorldState:
        # Make a deep copy to avoid mutating original
        updated = state.model_copy(deep=True)
        
        # Update each agent based on their activity
        for agent in updated.agents:
            energy = agent.get_attribute("energy", default=80, unit="%")
            
            if agent.activity == "work":
                energy.value = max(0.0, float(energy.value) - 10)  # Working drains energy
            else:  # resting
                energy.value = min(100.0, float(energy.value) + 15)  # Resting recovers energy
        
        updated.tick = tick
        return updated
    
    def validate_action(self, action: AgentAction, state: WorldState) -> bool:
        # Accept all actions for this simple example
        return True

print("Physics defined: SimpleWorkshopRules")
print("  - Working drains 10 energy per tick")
print("  - Resting recovers 15 energy per tick")

### 3.2 Set up the simulation

Now we'll create agents, cognition, and the orchestrator.

In [ ]:
from miniverse import Orchestrator, AgentProfile, AgentStatus, WorldState, ResourceState, EnvironmentState
from datetime import datetime, timezone

# Create agent profile
alice_profile = AgentProfile(
    agent_id="alice",
    name="Alice",
    age=28,
    background="Workshop technician",
    role="worker",
    personality="diligent and thoughtful",
    skills={"task_management": "proficient"},
    goals=["Maintain energy", "Complete tasks"],
    relationships={}
)

# Create initial world state
initial_state = WorldState(
    tick=0,
    timestamp=datetime.now(timezone.utc),
    environment=EnvironmentState(metrics={}),
    resources=ResourceState(
        metrics={
            "task_backlog": Stat(value=12, label="Task Backlog")
        }
    ),
    agents=[
        AgentStatus(
            agent_id="alice",
            display_name="Alice",
            attributes={"energy": Stat(value=80, unit="%", label="Energy")}
        )
    ]
)

# Create cognition with scratchpad so we can see state
alice_cognition = AgentCognition(
    executor=ThresholdExecutor(),
    scratchpad=Scratchpad()
)

# Create orchestrator
orchestrator = Orchestrator(
    world_state=initial_state,
    agents={"alice": alice_profile},
    world_prompt="",
    agent_prompts={"alice": "You are Alice, a workshop worker."},
    simulation_rules=SimpleWorkshopRules(),
    agent_cognition={"alice": alice_cognition}
)

print("Simulation ready!")
print(f"  Agents: 1 (Alice)")
print(f"  Physics: SimpleWorkshopRules")
print(f"  Cognition: ThresholdExecutor with Scratchpad")

### 3.3 Run the simulation and inspect each tick

Let's run 5 ticks and see what happens at each step.

In [ ]:
# Run simulation
result = await orchestrator.run(num_ticks=5)

print("\n" + "="*60)
print("SIMULATION COMPLETE - Let's examine what happened")
print("="*60)

# Get final state
final_state = result["final_state"]
alice_final = next(a for a in final_state.agents if a.agent_id == "alice")

print(f"\nFinal tick: {final_state.tick}")
print(f"Alice's final energy: {alice_final.get_attribute('energy').value}%")
print(f"Alice's final activity: {alice_final.activity}")

# Show scratchpad state if any
print(f"\nScratchpad state: {alice_cognition.scratchpad.state}")

print("\nKey observations:")
print("  - Each tick, physics was applied first (energy changed)")
print("  - Then executor decided what to do (work or rest)")
print("  - Action affected next tick's state")
print("  - This is the core simulation loop!")

### 3.4 Examine tick-by-tick state transitions

Let's run again and manually inspect each tick's state.

In [ ]:
# Reset simulation
initial_state = WorldState(
    tick=0,
    timestamp=datetime.now(timezone.utc),
    environment=EnvironmentState(metrics={}),
    resources=ResourceState(
        metrics={"task_backlog": Stat(value=12, label="Task Backlog")}
    ),
    agents=[
        AgentStatus(
            agent_id="alice",
            display_name="Alice",
            attributes={"energy": Stat(value=80, unit="%", label="Energy")}
        )
    ]
)

alice_cognition = AgentCognition(
    executor=ThresholdExecutor(),
    scratchpad=Scratchpad()
)

orchestrator = Orchestrator(
    world_state=initial_state,
    agents={"alice": alice_profile},
    world_prompt="",
    agent_prompts={"alice": "You are Alice."},
    simulation_rules=SimpleWorkshopRules(),
    agent_cognition={"alice": alice_cognition}
)

# Run tick by tick
print("\nTick-by-tick state transitions:\n")

for tick in range(1, 6):
    result = await orchestrator.run(num_ticks=1)
    state = result["final_state"]
    alice = next(a for a in state.agents if a.agent_id == "alice")
    energy = alice.get_attribute("energy").value
    backlog = state.resources.get_metric("task_backlog").value
    
    print(f"Tick {tick}:")
    print(f"  Energy: {energy}%")
    print(f"  Backlog: {int(backlog)} tasks")
    print(f"  Activity: {alice.activity}")
    print()

## Part 4: Adding LLM Intelligence

Now let's replace our threshold executor with an LLM that makes intelligent decisions.

**Note:** This requires LLM configuration (API keys).

In [ ]:
import os

# Check if LLM is configured
provider = os.getenv("LLM_PROVIDER")
model = os.getenv("LLM_MODEL")

if provider and model:
    print(f"✓ LLM configured: {provider}/{model}")
    print("  We can proceed with LLM examples")
else:
    print("✗ LLM not configured")
    print("\nTo use LLM examples, set:")
    print("  export LLM_PROVIDER=openai")
    print("  export LLM_MODEL=gpt-4")
    print("  export OPENAI_API_KEY=your_key")
    print("\nSkipping LLM examples for now...")

### 4.1 LLM Executor - Context-aware decisions

Let's see what context the LLM receives and how it decides.

In [ ]:
# Only run if LLM is configured
if provider and model:
    from miniverse.cognition import LLMExecutor
    
    # Create LLM-powered cognition
    alice_llm_cognition = AgentCognition(
        executor=LLMExecutor(),
        scratchpad=Scratchpad()
    )
    
    # Create simulation with LLM executor
    initial_state = WorldState(
        tick=0,
        timestamp=datetime.now(timezone.utc),
        environment=EnvironmentState(metrics={}),
        resources=ResourceState(
            metrics={"task_backlog": Stat(value=12, label="Task Backlog")}
        ),
        agents=[
            AgentStatus(
                agent_id="alice",
                display_name="Alice",
                attributes={"energy": Stat(value=80, unit="%", label="Energy")}
            )
        ]
    )
    
    orchestrator_llm = Orchestrator(
        world_state=initial_state,
        agents={"alice": alice_profile},
        world_prompt="",
        agent_prompts={
            "alice": """You are Alice, a thoughtful workshop worker.
            
Your goals:
- Maintain your wellbeing (don't let energy drop too low)
- Address the task backlog when it's high
- Balance productivity and self-care

Available actions: work, rest"""
        },
        simulation_rules=SimpleWorkshopRules(),
        agent_cognition={"alice": alice_llm_cognition},
        llm_provider=provider,
        llm_model=model
    )
    
    print("Running 3 ticks with LLM executor...\n")
    result = await orchestrator_llm.run(num_ticks=3)
    
    final = result["final_state"]
    alice = next(a for a in final.agents if a.agent_id == "alice")
    
    print(f"\nLLM simulation complete:")
    print(f"  Final energy: {alice.get_attribute('energy').value}%")
    print(f"  Final activity: {alice.activity}")
    print(f"\nNotice: LLM makes nuanced decisions, not just threshold-based!")
else:
    print("Skipping LLM example (not configured)")

## Part 5: Memory and Persistence

Let's examine how memories accumulate and how to inspect them.

### 5.1 Accessing memories from persistence

By default, Miniverse uses in-memory persistence. Let's inspect what's stored.

In [ ]:
# Get the simulation ID from last run
sim_id = result["simulation_id"]
print(f"Simulation ID: {sim_id}")

# Access persistence (in-memory by default)
persistence = orchestrator.persistence

# Get all memories for Alice
alice_memories = await persistence.get_memories(sim_id, agent_id="alice")

print(f"\nAlice has {len(alice_memories)} memories:")
for i, memory in enumerate(alice_memories[:5], 1):  # Show first 5
    print(f"\n  Memory {i} (Tick {memory.tick}):")
    print(f"    Type: {memory.memory_type}")
    print(f"    Importance: {memory.importance}")
    print(f"    Content: \"{memory.content[:80]}...\" if len(memory.content) > 80 else memory.content")

### 5.2 Understanding memory retrieval

The `MemoryStrategy` controls how agents retrieve relevant memories.

In [ ]:
from miniverse.memory import SimpleMemoryStream

# Create a memory stream
memory_stream = SimpleMemoryStream()

# Simulate some memories
test_memories = [
    AgentMemory(
        agent_id="alice",
        tick=1,
        timestamp=datetime.now(timezone.utc),
        content="Started working on authentication feature",
        memory_type="observation",
        importance=5,
        tags=["work", "coding"]
    ),
    AgentMemory(
        agent_id="alice",
        tick=2,
        timestamp=datetime.now(timezone.utc),
        content="Helped Bob debug a tricky issue - felt good to collaborate",
        memory_type="observation",
        importance=7,
        tags=["collaboration", "teamwork", "Bob"]
    ),
    AgentMemory(
        agent_id="alice",
        tick=3,
        timestamp=datetime.now(timezone.utc),
        content="Energy getting low, need a break soon",
        memory_type="observation",
        importance=6,
        tags=["energy", "wellbeing"]
    ),
    AgentMemory(
        agent_id="alice",
        tick=5,
        timestamp=datetime.now(timezone.utc),
        content="I've been balancing work and rest well - staying productive without burning out",
        memory_type="reflection",
        importance=8,
        tags=["reflection", "balance", "goals"]
    ),
]

# Retrieve relevant memories for a query
query = "How is my energy doing?"
relevant = memory_stream.get_relevant_memories(
    memories=test_memories,
    query=query,
    limit=3
)

print(f"Query: '{query}'")
print(f"\nRelevant memories (top 3):")
for i, memory in enumerate(relevant, 1):
    print(f"\n  {i}. (Tick {memory.tick}, importance {memory.importance}):")
    print(f"     \"{memory.content}\"")

## Part 6: Planning - Multi-step reasoning

Now let's add planning capability to agents.

### 6.1 Create a simple deterministic planner

In [ ]:
from miniverse.cognition import Plan, PlanStep

class DailyRoutinePlanner:
    """Creates a fixed daily routine plan."""
    
    async def generate_plan(self, agent_id, agent_profile, world_state, recent_memories, *, context):
        # Create a simple 4-step plan
        plan = Plan(
            steps=[
                PlanStep(description="Morning check-in and review tasks", metadata={"duration": 1}),
                PlanStep(description="Work on high-priority tasks", metadata={"duration": 3}),
                PlanStep(description="Take lunch break and rest", metadata={"duration": 1}),
                PlanStep(description="Afternoon work session", metadata={"duration": 2}),
            ],
            current_step_index=0
        )
        return plan, "Daily routine plan"

# Test it
planner = DailyRoutinePlanner()
plan, reasoning = await planner.generate_plan(
    agent_id="alice",
    agent_profile=alice_profile,
    world_state=initial_state,
    recent_memories=[],
    context={}
)

print("Generated Plan:")
print(f"  Reasoning: {reasoning}")
print(f"  Total steps: {len(plan.steps)}")
print(f"\n  Steps:")
for i, step in enumerate(plan.steps):
    duration = step.metadata.get("duration", "?")
    print(f"    {i}. {step.description} ({duration} ticks)")

### 6.2 Run simulation with planning

Let's see how plans guide agent behavior over multiple ticks.

In [ ]:
# Create executor that follows plan
class PlanFollowerExecutor:
    """Executor that follows the current plan step."""
    
    async def choose_action(self, agent_id, perception, scratchpad, *, plan, plan_step, context):
        if plan_step:
            # We have a plan - follow it
            if "work" in plan_step.description.lower():
                action_type = "work"
            elif "break" in plan_step.description.lower() or "rest" in plan_step.description.lower():
                action_type = "rest"
            else:
                action_type = "work"  # Default
            
            reasoning = f"Following plan: {plan_step.description}"
        else:
            # No plan - fall back to simple logic
            action_type = "rest"
            reasoning = "No plan - resting"
        
        return AgentAction(
            agent_id=agent_id,
            tick=perception.tick,
            action_type=action_type,
            reasoning=reasoning
        )

# Create cognition with planner
alice_planned_cognition = AgentCognition(
    executor=PlanFollowerExecutor(),
    planner=DailyRoutinePlanner(),
    scratchpad=Scratchpad()
)

# Run simulation
initial_state = WorldState(
    tick=0,
    timestamp=datetime.now(timezone.utc),
    environment=EnvironmentState(metrics={}),
    resources=ResourceState(
        metrics={"task_backlog": Stat(value=10, label="Task Backlog")}
    ),
    agents=[
        AgentStatus(
            agent_id="alice",
            display_name="Alice",
            attributes={"energy": Stat(value=100, unit="%", label="Energy")}
        )
    ]
)

orchestrator_planned = Orchestrator(
    world_state=initial_state,
    agents={"alice": alice_profile},
    world_prompt="",
    agent_prompts={"alice": "You follow your daily routine plan."},
    simulation_rules=SimpleWorkshopRules(),
    agent_cognition={"alice": alice_planned_cognition}
)

print("Running 7 ticks with planner...\n")
result = await orchestrator_planned.run(num_ticks=7)

print("\nSimulation with planning complete!")
print("\nKey insight: Agent followed multi-step plan across ticks")
print("  - Plan persisted in scratchpad")
print("  - Executor used current plan step to decide actions")
print("  - Plan advanced automatically each tick")

## Part 7: Reflection - Learning from experience

Reflection engines analyze accumulated memories and generate insights.

### 7.1 Simple heuristic reflection

In [ ]:
class SimpleReflection:
    """Heuristic reflection: detects patterns in memories."""
    
    async def maybe_reflect(
        self,
        agent_id,
        agent_profile,
        recent_memories,
        world_state,
        *,
        context
    ):
        # Only reflect if we have enough memories
        if len(recent_memories) < 5:
            return []
        
        # Count "work" vs "rest" mentions
        work_count = sum(1 for m in recent_memories if "work" in m.content.lower())
        rest_count = sum(1 for m in recent_memories if "rest" in m.content.lower())
        
        reflections = []
        
        if work_count > rest_count * 2:
            reflections.append(
                AgentMemory(
                    agent_id=agent_id,
                    tick=world_state.tick,
                    timestamp=datetime.now(timezone.utc),
                    content="I've been working a lot lately - might need more rest to avoid burnout.",
                    memory_type="reflection",
                    importance=8,
                    tags=["reflection", "balance", "wellbeing"]
                )
            )
        
        return reflections

# Test reflection
test_memories = [
    AgentMemory(agent_id="alice", tick=i, timestamp=datetime.now(timezone.utc),
                content="Worked on tasks" if i % 2 == 0 else "Took a break",
                memory_type="observation", importance=5, tags=[])
    for i in range(10)
]

reflector = SimpleReflection()
reflections = await reflector.maybe_reflect(
    agent_id="alice",
    agent_profile=alice_profile,
    recent_memories=test_memories,
    world_state=initial_state,
    context={}
)

print("Reflection Analysis:")
print(f"  Analyzed {len(test_memories)} memories")
print(f"  Generated {len(reflections)} reflections")
if reflections:
    for reflection in reflections:
        print(f"\n  Reflection:")
        print(f"    Content: \"{reflection.content}\"")
        print(f"    Importance: {reflection.importance}")
        print(f"    Tags: {reflection.tags}")

## Summary: What we've learned

You now understand:

**Data Structures:**
- `Stat` - Flexible metrics with units/labels
- `AgentProfile` - WHO the agent is (identity, personality, goals)
- `AgentStatus` - CURRENT state (activity, location, attributes)
- `WorldState` - Complete simulation state
- `Plan` - Multi-step plans with duration
- `Scratchpad` - Working memory (key-value store)
- `AgentMemory` - Long-term memories with importance/tags

**Cognition Modules:**
- `Executor` - Required: chooses actions each tick
- `Planner` - Optional: generates multi-step plans
- `ReflectionEngine` - Optional: synthesizes insights from memories
- `AgentCognition` - Bundles modules together

**Simulation Mechanics:**
- `SimulationRules` - Deterministic physics (world updates)
- `Orchestrator` - Main simulation loop
- Tick loop: Physics → Plan → Perceive → Execute → Update

**Design Patterns:**
- Minimal: `AgentCognition(executor=MyExecutor())`
- With planning: Add `planner` and `scratchpad`
- With reflection: Add `reflection` engine
- LLM-driven: Use `LLMExecutor`, `LLMPlanner`, `LLMReflectionEngine`

**Next steps:**
- Explore the workshop examples (01-05) in `examples/workshop/`
- Try LLM-powered agents (configure API keys)
- Build your own simulation domain
- Experiment with custom executors, planners, physics